# סוכני Azure AI עם תמיכה ב-Model Context Protocol (MCP)

מחברת זו מדגימה כיצד להשתמש בסוכני Azure AI עם כלים של Model Context Protocol (MCP) כדי ליצור סוכן חכם שיכול לנצל שרתי MCP חיצוניים לשיפור היכולות.


## התקנת חבילות NuGet נדרשות

ראשית, עלינו להתקין את חבילת Azure AI Agents Persistent שמספקת את הפונקציונליות המרכזית לעבודה עם Azure AI Agents.


## יתרונות אימות ללא מפתחות

מחברת זו מדגימה **אימות ללא מפתחות**, שמציע מספר יתרונות:
- ✅ **אין צורך לנהל מפתחות API** - משתמש באימות מבוסס זהות של Azure
- ✅ **אבטחה משופרת** - אין סודות שמאוחסנים בקוד או בקובץ הגדרות
- ✅ **סבב אישורים אוטומטי** - Azure מטפל במחזור החיים של האישורים
- ✅ **גישה מבוססת תפקידים** - משתמש ב-RBAC של Azure להרשאות מדויקות

ה-`DefaultAzureCredential` ישתמש באופן אוטומטי במקור האישורים הטוב ביותר הזמין:
1. Managed Identity (כאשר פועל ב-Azure)
2. אישורי Azure CLI (במהלך פיתוח)
3. אישורי Visual Studio
4. משתני סביבה (אם הוגדרו)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

התקן את חבילת Azure Identity לאימות עם שירותי Azure באמצעות DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## ייבוא מרחבי שמות נדרשים

ייבא את מרחבי השמות הדרושים עבור Azure AI Agents ו-Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## הגדרת Azure AI Agent Client (אימות ללא מפתח)

הגדר את משתני התצורה ויצירת PersistentAgentsClient באמצעות **אימות ללא מפתח**:
- **projectEndpoint**: נקודת הקצה של פרויקט Azure AI Foundry  
- **modelDeploymentName**: שם המודל הבינה המלאכותית שפורסם (GPT-4.1 nano)  
- **mcpServerUrl**: כתובת ה-URL של שרת MCP (Microsoft Learn API)  
- **mcpServerLabel**: תווית לזיהוי שרת ה-MCP  
- **DefaultAzureCredential**: משתמש בזהות מנוהלת, Azure CLI או מקורות אישור אחרים (ללא צורך במפתחות API)  


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## יצירת הגדרת כלי MCP

צור הגדרת כלי MCP שמתחברת לשרת MCP של Microsoft Learn. פעולה זו תאפשר לסוכן לגשת לתוכן ולתיעוד של Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## יצירת סוכן AI

צרו סוכן AI מתמשך עם המודל וכלי MCP שצוינו. הסוכן מוגדר עם:
- מודל GPT-4.1 nano  
- הנחיות לשימוש בכלי MCP לצורך סיוע  
- גישה לשרת MCP של Microsoft Learn  


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## יצירת שרשור ושליחת הודעה

צור שרשור שיחה ושלח הודעת משתמש ששואלת על ההבדל בין Azure OpenAI ל-OpenAI. זה יבחן את יכולת הסוכן להשתמש בכלי MCP כדי לספק מידע מדויק.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## הגדרת משאבי כלי MCP (ללא מפתח)

הגדר את משאבי כלי MCP. לגישה ללא מפתח אמיתית, ניתן להסיר כותרות מותאמות אישית אם שרת MCP תומך באימות מבוסס זהות של Azure. הדוגמה למטה מראה כיצד להוסיף כותרות במקרה הצורך, אך עבור תרחישים ללא מפתח, ייתכן שלא יהיה צורך בכך.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## התחלת הפעלת סוכן

צור והפעל תהליך לעיבוד הודעת המשתמש. הסוכן ישתמש בכלים ובמשאבים שהוגדרו ב-MCP כדי ליצור תגובה.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## ניטור הרצה וטיפול באישורי כלים (ללא מפתח)

נטר את מצב ההרצה של הסוכן וטפל בכל אישור כלי נדרש. הלולאה הזו:
1. ממתינה לסיום ההרצה או לפעולה נדרשת
2. מאשרת באופן אוטומטי קריאות לכלי MCP כאשר נדרש
3. עבור אימות ללא מפתח, ייתכן שאין צורך בכותרות אם שרת MCP תומך בזהות Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## הצגת תוצאות השיחה

שליפת והצגת כל ההודעות בשיחה, כולל שאלות המשתמש ותשובות הנציג. ההודעות מוצגות בסדר כרונולוגי עם חותמות זמן וסימוני תפקיד.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). בעוד שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
